In [1]:
%%timeit

from pathlib import Path

inp = Path("input.txt").read_text().strip().splitlines()
inp = [(int((split := line.split(": "))[0]), list(map(int, split[1].split()))) for line in inp]

def possible1(target, args):
    if len(args) == 1:
        return target == args[0]

    *init, last = args
    return possible1(target - last, init) or (not target % last and possible1(target / last, init))


def possible2(target, args):
    if len(args) == 1:
        return target == args[0]

    *init, last = args
    return (
        possible2(target - last, init)
        or (not target % last and possible2(target / last, init))
        or (not (uncat := (target - last) / 10**len(str(last))) % 1 and possible2(uncat, init))
    )


print(sum(target for target, args in inp if possible1(target, args)))
print(sum(target for target, args in inp if possible2(target, args)))

12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396350192
106016735664498
12940396

In [ ]:
%%timeit

from math import log10
# from pathlib import Path

# HOME = Path(__file__).parent

# hyperfine "pypy --jit max_unroll_recursion=0 p2.py" --warmup 5

def unconc(a: int, b: int):
    x = int(log10(b)) + 1
    q,r = divmod(a, x)
    return q if r == b else -1

def try_fix(curr: int, parts: tuple[int, ...], i: int):
    # Do this first, as triggered 19654 times
    if i < 0:
        return curr == 0
    # Do this second, as triggered 18960 times (0.964)
    if curr < 0:
        return False

    a = parts[i]
    q,r = divmod(curr, a)
    return (
        not r and try_fix(q, parts, i - 1)
        or try_fix(unconc(curr, a), parts, i - 1)
        or try_fix(curr - a, parts, i - 1)
    )


with open("input.txt") as f: # HOME / "input.txt"
    total = 0
    for line in f:
        _res, _parts = line.split(":")
        parts = tuple(map(int, _parts.split()))
        res = int(_res)
        if try_fix(res, parts, len(parts) - 1):
            total += res
    print(total)


In [ ]:
%%timeit

from math import ceil, log10

def equation(line):
  res, vals = line.split(": ")
  return (int(res), tuple(map(int, vals.split(" "))))

def order(x):
  return 10**ceil(log10(x + 1))

def valid(eq):
  match (eq):
    case (0, ()): return True
    case (_, ()): return False
    case (res, (*init, last)): return (
      ((res % last == 0) and valid((res//last, init))) or
      ((res >= last) and valid((res - last, init))) or
      ((res % order(last) == last) and valid((res//order(last), init)))
    )

lines = filter(bool, open("input.txt", "r").read().split("\n"))
print(sum(map(lambda x: x[0], filter(valid, map(equation, lines)))))

In [ ]:
%%timeit

def endswith(a, b):
    return a % (10 ** (int(log10(b)) + 1)) == b

def unconcat(a, b):
    return a // (10 ** (int(log10(b)) + 1))

def solve(target, operands, part):
    *operands, operand = operands
    if not operands:
        return operand == target
    if (
        part == 2
        and endswith(target, operand)
        and solve(unconcat(target, operand), operands, part)
    ):
        return True
    if target % operand == 0 and solve(target // operand, operands, part):
        return True
    return solve(target - operand, operands, part)

def part_one():
    return sum(target for target, *operands in DATA if solve(target, operands, 1))

def part_two():
    return sum(target for target, *operands in DATA if solve(target, operands, 2))